## Initialize Modules and Pathways

In [ ]:

''' Need very specifc pytorch setup to work on Ubuntu system:
    pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio -f https://download.pytorch.org/whl/torch_stable.html
'''

# add location of Cellpose3D to system paths
import os
import sys
import time
sys.path.append('/home/elyse/Documents/py_scripts/')
sys.path.append('/home/elyse/Documents/py_scripts/Cellpose3D')
    
from Cellpose3D.utils import h5_converter, csv_generator
from glob import glob
    
# setup paths
# need to have created masks 
data_root = '/home/elyse/Documents/GitHub/LS_evaluation_tool/'
folders = ['test_data/images/annotated/block_4', 'test_data/cellpose_output/flow']
files = ['block_4_signal.tif', 'block_4_signal_cp_masks.tif']
output_path = os.path.join(data_root, folders[0], 'cellpose3D_output')
    
#check if output path exists and create if necessary
if not os.path.exists(output_path):
    os.mkdir(output_path)

# time preprocessing
start_processing = time.time()

# format tif stacks as h5
h5_converter.prepare_images(data_path = data_root, folders = [folders[0]], identifier = files[0])
h5_converter.prepare_masks(data_path = data_root, folders = [folders[1]], identifier = files[1])
    
# get new .h5s and create file lists\n",
img_h5 = glob(os.path.join(data_root, folders[0], '*.h5'))[0]
mask_h5 = glob(os.path.join(data_root, folders[1], '*.h5'))[0]
csv_generator.create_csv(data_list = [(img_h5, mask_h5)], save_path = output_path)

preprocess_time = time.time() - start_processing
print('Network input files created: {0} seconds.'.format(preprocess_time))

[12:38:52] Processing image 1/1 in "test_data/images/annotated/block_4"
[12:38:53] Processing mask 1/1 in test_data/cellpose_output/flow


In [ ]:
# run cellpose3D network"
start_model = time.time()
test_path = '/home/elyse/Documents/py_scripts/Cellpose3D/epoch=999.ckpt'

!python /home/elyse/Documents/py_scripts/Cellpose3D/apply_network.py --output_path {output_path} --ckpt_path {test_path}

model_time = time.time() - start_model
print('Model Completed: {0} seconds '.format(preprocess_time + model_time))